<a href="https://colab.research.google.com/github/Bar89bas/Home_Sales-Project/blob/main/home_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [962 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,230 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,059 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,254 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,079 kB]
Hit:13 https://ppa.launc

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql(""" select date, bedrooms, mean(price) from home_sales where bedrooms=4 group by 1,2""").show()


+----------+--------+------------------+
|      date|bedrooms|       mean(price)|
+----------+--------+------------------+
|2021-10-04|       4|          299210.0|
|2022-01-21|       4|         235127.75|
|2021-07-28|       4|          318108.4|
|2020-11-24|       4|         296285.25|
|2021-01-29|       4|231625.66666666666|
|2021-05-06|       4|        345576.125|
|2021-08-06|       4| 313615.8333333333|
|2020-12-21|       4| 266941.5833333333|
|2020-06-27|       4|          265233.0|
|2020-10-15|       4|          256773.0|
|2020-02-09|       4|221187.22222222222|
|2019-08-04|       4|308513.14285714284|
|2019-09-29|       4|          286234.2|
|2021-05-25|       4|          328821.0|
|2019-12-22|       4|234412.33333333334|
|2019-07-30|       4|223573.66666666666|
|2020-09-22|       4|          277209.8|
|2022-01-06|       4|         274989.75|
|2020-10-18|       4|          283202.0|
|2020-01-23|       4|          278643.1|
+----------+--------+------------------+
only showing top

In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql(""" select date, bedrooms, bathrooms, mean(price) from home_sales where (bedrooms=3 and bathrooms =3) group by 1,2,3""").show()

+----------+--------+---------+------------------+
|      date|bedrooms|bathrooms|       mean(price)|
+----------+--------+---------+------------------+
|2019-02-20|       3|        3|          266404.0|
|2019-12-15|       3|        3|          282642.0|
|2020-04-02|       3|        3| 317168.3333333333|
|2021-05-11|       3|        3|          264984.2|
|2021-09-11|       3|        3|          230642.0|
|2021-08-10|       3|        3|          291059.0|
|2020-12-18|       3|        3|          563429.0|
|2021-10-17|       3|        3|          283445.0|
|2022-05-06|       3|        3| 298931.1666666667|
|2019-04-19|       3|        3|292636.85714285716|
|2019-02-25|       3|        3|          177283.0|
|2020-08-06|       3|        3| 362263.6666666667|
|2021-05-09|       3|        3|          163684.0|
|2021-10-03|       3|        3| 278359.6666666667|
|2019-04-06|       3|        3|          124452.0|
|2019-09-21|       3|        3| 308464.6666666667|
|2019-11-15|       3|        3|

In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql(""" select date, bedrooms, bathrooms,floors, sqft_lot, mean(price) from home_sales where (bedrooms=3 and bathrooms =3) and (floors= 2 and sqft_lot >= 2000) group by 1,2,3,4,5""").show()


+----------+--------+---------+------+--------+-----------+
|      date|bedrooms|bathrooms|floors|sqft_lot|mean(price)|
+----------+--------+---------+------+--------+-----------+
|2021-08-21|       3|        3|     2|   11084|   209252.0|
|2020-10-31|       3|        3|     2|   12695|   447944.0|
|2021-05-26|       3|        3|     2|    8715|   282679.0|
|2022-01-28|       3|        3|     2|    9408|   280429.0|
|2020-08-06|       3|        3|     2|   12093|   439341.0|
|2019-02-22|       3|        3|     2|    9015|   293388.0|
|2021-04-04|       3|        3|     2|   14895|   137532.0|
|2019-10-19|       3|        3|     2|   13801|   351494.0|
|2019-11-09|       3|        3|     2|    8027|   276309.0|
|2019-06-01|       3|        3|     2|   10765|   320070.0|
|2019-08-25|       3|        3|     2|   11386|   317951.0|
|2020-03-03|       3|        3|     2|   12848|   318015.0|
|2022-04-06|       3|        3|     2|   11475|   343624.0|
|2020-05-01|       3|        3|     2|  

In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from home_sales group by view having ROUND(AVG(price),2) >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  81|          1053472.79|
|  76|          1058802.78|
|  91|          1137372.73|
|  93|          1026006.06|
|  52|           733780.26|
|  86|          1070444.25|
|  94|           1033536.2|
|  57|            734340.5|
|  54|           798684.82|
|  96|          1017815.92|
|  92|           970402.55|
|  64|           767036.67|
|  61|           746877.59|
|  88|          1031719.35|
|  72|           780914.67|
|  59|            791453.0|
+----+--------------------+
only showing top 20 rows

--- 0.6465420722961426 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from home_sales group by view having ROUND(AVG(price),2) >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  81|          1053472.79|
|  76|          1058802.78|
|  91|          1137372.73|
|  93|          1026006.06|
|  52|           733780.26|
|  86|          1070444.25|
|  94|           1033536.2|
|  57|            734340.5|
|  54|           798684.82|
|  96|          1017815.92|
|  92|           970402.55|
|  64|           767036.67|
|  61|           746877.59|
|  88|          1031719.35|
|  72|           780914.67|
|  59|            791453.0|
+----+--------------------+
only showing top 20 rows

--- 0.320141077041626 seconds ---


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.parquet('home_parquet', mode='overwrite')

In [14]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet('home_parquet')

In [15]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('parquet_temp_home')

In [16]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from parquet_temp_home group by view having ROUND(AVG(price),2) < 350000").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  31|           287988.84|
|  34|           286124.07|
|  28|           285474.25|
|  26|           281224.95|
|  27|            281702.6|
|  44|           282856.44|
|  12|           286575.22|
|  22|           284908.42|
|  47|           292925.62|
|   1|           288248.11|
|  13|           285849.95|
|   6|           284076.65|
|  16|           291990.83|
|   3|           284314.53|
|  40|           290768.72|
|  20|           284820.43|
|  48|           283862.99|
|   5|           278096.94|
|  19|           280347.62|
|  41|           288697.21|
+----+--------------------+
only showing top 20 rows

--- 0.5019409656524658 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [18]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False